# **Project 3 Group 4 Problem 2**

In [118]:
# Implementing Mini Block Cipher with key size 16 bit and block size 16 bit (Task 1)
# Includes: S-box, ShiftRows, Mix, AddRoundKey, Key expansion,
# encrypt_round1/2, decrypt_round2/1, encrypt, decrypt,generation of plaintext-ciphertext pairs
# MITM helpers, and MITM demo

In [119]:
# =============================
# Cell 1 - Imports and Utilities
# =============================
import random
from collections import defaultdict

def split_words(x):
    "Splitting a 16-bit size word into 4 nibbles"

    return [ (x >> 12) & 0xF, (x >> 8) & 0xF, (x >> 4) & 0xF, x & 0xF ]

def join_nibbles(nibs):
    "Joining 4 nibbles into a 16-bit integer"
    return ((nibs[0] & 0xF) << 12) | ((nibs[1] & 0xF) << 8) | ((nibs[2] & 0xF) << 4) | (nibs[3] & 0xF)

In [120]:
# =============================
# Cell 2 - S-box and inverse S-box
# =============================

# 4-bit S-box (SAES)
SBOX = [0x9,0x4,0xA,0xB,0xD,0x1,0x8,0x5,0x6,0x2,0x0,0x3,0xC,0xE,0xF,0x7]
INV_SBOX = [0]*16
for i,v in enumerate(SBOX):
    INV_SBOX[v] = i

def substitute(state):
    "Applying S-box to each nibble"
    return [SBOX[x & 0xF] for x in state]

def inv_substitute(state):
    "Applying the inverse of S-box to each nibble"
    return [INV_SBOX[x & 0xF] for x in state]


In [121]:
# =============================
# Cell 3 - ShiftRows and inverse ShiftRows
# =============================

def shiftrows(state):
    "Swapping s2 and s3 in order to do the shiftrow"
    return [state[0], state[1], state[3], state[2]]

def inv_shiftrows(state):
    "Applying the inverse of the shiftrows"
    return [state[0], state[1], state[3], state[2]]

In [122]:
# =============================
# Cell 4 - MixColumns, Inverse MixColumns, and GF(16) Multiplication
# =============================

# GF(16) multiplication with modulo x^4 + x + 1 for SAES 
def gf_multiplication(a, b):
    p = 0
    for i in range(4):               
        if (b >> i) & 1:
            p ^= (a << i)
    for i in range(7, 3, -1):        #  reduce modulo 0x13
        if (p >> i) & 1:
            p ^= 0x13 << (i - 4)
    return p & 0xF

# SAES Mix + InvMix (2x2) 
def mix(state):
    "SAES MixColumns - matrix [[1,4],[4,1]] over GF(16)"
    s0, s1, s2, s3 = [x & 0xF for x in state]
    temp0 = (s0 ^ gf_multiplication(4, s2)) & 0xF
    temp2 = (gf_multiplication(4, s0) ^ s2) & 0xF
    temp1 = (s1 ^ gf_multiplication(4, s3)) & 0xF
    temp3 = (gf_multiplication(4, s1) ^ s3) & 0xF
    return [temp0, temp1, temp2, temp3]

def inv_mix(state):
    "Inverse MixColumns - matrix [[1,4],[4,1]] over GF(16)"
    s0, s1, s2, s3 = [x & 0xF for x in state]
    temp0 = (gf_multiplication(9, s0) ^ gf_multiplication(2, s2)) & 0xF
    temp2 = (gf_multiplication(2, s0) ^ gf_multiplication(9, s2)) & 0xF
    temp1 = (gf_multiplication(9, s1) ^ gf_multiplication(2, s3)) & 0xF
    temp3 = (gf_multiplication(2, s1) ^ gf_multiplication(9, s3)) & 0xF
    return [temp0, temp1, temp2, temp3]

In [123]:
# =============================
# Cell 5 - AddRoundKey
# =============================

def add_round_key(state, round_key):
    "XORing the 4 nibbles with the round_key"
    key_nibs = split_words(round_key)
    return [(state[i] ^ key_nibs[i]) & 0xF for i in range(4)]

In [124]:
# =============================
# Cell 6 - Key expansion
# =============================

def key_expansion(K):
    # Splitting 16-bit key into two 8-bit words
    word0 = (K >> 8) & 0xFF
    word1 = K & 0xFF

    # Making sure to round constants
    RCONST1 = 0x80  # 1000 0000
    RCONST2 = 0x30  # 0011 0000

    # Swapping nibbles in an 8-bit word
    def Rotate_Nibbles(b):  
        return ((b << 4) | (b >> 4)) & 0xFF

    # Applying 4-bit SBOX to each of the nibbles
    def Sub_Nibbles(b):  
        high_nibble = SBOX[(b >> 4) & 0xF]
        low_nibble = SBOX[b & 0xF]
        return ((high_nibble << 4) | low_nibble) & 0xFF

    # Using the SAES schedule
    word2 = word0 ^ RCONST1 ^ Sub_Nibbles(Rotate_Nibbles(word1))
    word3 = word2 ^ word1
    word4 = word2 ^ RCONST2 ^ Sub_Nibbles(Rotate_Nibbles(word3))
    word5 = word3 ^ word4

    # Rounding keys for the two round ciphers
    K1 = (word2 << 8) | word3
    K2 = (word4 << 8) | word5
    return K1, K2


In [125]:
# =============================
# Cell 7 - Round functions
# =============================

# Two rounds of encryption to get the ciphertext.
def encrypt_round1(state_bits, K1):
    "encrypt_round1() using K1 -> i. Subsititute(); ii. Shift(); iii. Mix(); iv. AddRoundKey()"
    s = split_words(state_bits)
    s = substitute(s)
    s = shiftrows(s)
    s = mix(s)
    s = add_round_key(s, K1)
    return join_nibbles(s)

def encrypt_round2(state_bits, K2):
    "encrypt_round2() using K2 -> i. Subsititute(); ii. Shift() ; iii. AddRoundKey()"
    s = split_words(state_bits)
    s = substitute(s)
    s = shiftrows(s)
    s = add_round_key(s, K2)
    return join_nibbles(s)

# Two rounds of decryption to recover the plaintext
def decrypt_round2(state_bits, K2):
    "decrypt_round2() using K2 -> i. AddRoundKey(); ii. Shift(); iii. Subsititute()"
    s = split_words(state_bits)
    s = add_round_key(s, K2)
    s = inv_shiftrows(s)
    s = inv_substitute(s)
    return join_nibbles(s)

def decrypt_round1(state_bits, K1):
    "decrypt_round1() using K1 -> i. AddRoundKey(); ii. Mix(); iii. Shift(); iv. Subsititute()"
    s = split_words(state_bits)
    s = add_round_key(s, K1)
    s = inv_mix(s)
    s = inv_shiftrows(s)
    s = inv_substitute(s)
    return join_nibbles(s)



In [126]:
# =============================
# Cell 8 - Full encryption and decryption 
# =============================

def encrypt(plaintext, K):
    "Encrypting a 16-bit plaintext with 16-bit key in order to get a 16-bit ciphertext"
    K1, K2 = key_expansion(K)
    intermediate_ciphertext = encrypt_round1(plaintext, K1)
    ciphertext = encrypt_round2(intermediate_ciphertext, K2)
    return ciphertext

def decrypt(ciphertext, K):
    "Decrypting a 16-bit ciphertext with key in order to get the 16-bit plaintext"
    K1, K2 = key_expansion(K)
    intermediate_plaintext = decrypt_round2(ciphertext, K2)
    plaintext = decrypt_round1(intermediate_plaintext, K1)
    return plaintext



In [127]:

# =============================
# Cell 9 - Create at least 10 plaintext-ciphertext pairs
# =============================

def create_pairs(n=12, seed=None, *, format='hex', ensure_unique=True):
    """Creation of at least ten distinct plaintext–ciphertext pairs using random 16-bit keys.

    Args:
        n - Number of pairs to generate.
        seed - Optional seed for reproducible results.
        format - Representation for the returned values - 'hex', 'int', or 'bin'.
        ensure_unique - Enforce unique (key, plaintext) combinations when True.

    Returns:
        list[dict[str, str | int]]: Generated pairs formatted per the selected representation.
    """
    if n <= 0:
        raise ValueError('n must be a positive integer')
    if format not in {'hex', 'int', 'bin'}:
        raise ValueError("format must be one of {'hex', 'int', 'bin'}")

    rng = random.Random(seed)
    pairs = []
    seen = set()

    formatter = {
        'hex': lambda value: f"{value:04X}",
        'bin': lambda value: f"{value:016b}",
        'int': lambda value: value,
    }[format]
    suffix = {'hex': 'hex', 'bin': 'bin', 'int': 'int'}[format]

    while len(pairs) < n:
        K = rng.randrange(0, 0x10000)
        P = rng.randrange(0, 0x10000)

        if ensure_unique:
            if (K, P) in seen:
                continue
            seen.add((K, P))

        C = encrypt(P, K)
        K1, K2 = key_expansion(K)
        pairs.append({
            f'key_{suffix}': formatter(K),
            f'K1_{suffix}': formatter(K1),
            f'K2_{suffix}': formatter(K2),
            f'plaintext_{suffix}': formatter(P),
            f'ciphertext_{suffix}': formatter(C),
        })

    return pairs


def create_pairs_hex(*args, **kwargs):
    """Compatibility helper that defaults to hexadecimal output.

    This makes it convenient to request the legacy hex format when additional
    args such as `ensure_unique` are provided.
    """
    kwargs.setdefault('format', 'hex')
    return create_pairs(*args, **kwargs)


In [128]:

# =============================
# Cell 10 -  Display of the created plaintext-ciphertext pairs
# =============================

# Creating 12 distinct plaintext–ciphertext pairs and displaying them in hex format
pairs_hex = create_pairs(12, seed=12345, ensure_unique=True)  # seed in order to keep repeating
for row in pairs_hex:
    print(row)

# Displaying the same pairs in binary format
print()
print('Sample pairs in binary format:')
for row in create_pairs(12, seed=54321, format='bin'):
    print(row)


{'key_hex': 'D54F', 'K1_hex': '2867', 'K2_hex': '4027', 'plaintext_hex': '0534', 'ciphertext_hex': '4B45'}
{'key_hex': '98E7', 'K1_hex': '47A0', 'K2_hex': 'E747', 'plaintext_hex': 'BCA0', 'ciphertext_hex': 'F450'}
{'key_hex': '6327', 'K1_hex': 'B99E', 'K2_hex': '7BE5', 'plaintext_hex': '8A5D', 'ciphertext_hex': 'E599'}
{'key_hex': 'DF59', 'K1_hex': '7E27', 'K2_hex': '1433', 'plaintext_hex': '52C8', 'ciphertext_hex': 'F02A'}
{'key_hex': 'BF02', 'K1_hex': '9694', 'K2_hex': '74E0', 'plaintext_hex': '3F9F', 'ciphertext_hex': '96CD'}
{'key_hex': 'DDA9', 'K1_hex': '7DD4', 'K2_hex': '9347', 'plaintext_hex': '85AD', 'ciphertext_hex': '5FD1'}
{'key_hex': '5943', 'K1_hex': '6427', 'K2_hex': '0E29', 'plaintext_hex': '5FA5', 'ciphertext_hex': 'B54C'}
{'key_hex': 'B5B5', 'K1_hex': '2693', 'K2_hex': 'A437', 'plaintext_hex': '2EBE', 'ciphertext_hex': 'E511'}
{'key_hex': 'D301', 'K1_hex': '1A1B', 'K2_hex': '1E05', 'plaintext_hex': '5549', 'ciphertext_hex': '7EA0'}
{'key_hex': '4BD9', 'K1_hex': 'E53C',

In [129]:
# =============================
# Cell 11 - Meet-in-the-middle helpers
# =============================

def double_round_encrypt(plaintext, K1, K2):
    """Encrypt a block using independent round keys."""
    return encrypt_round2(encrypt_round1(plaintext, K1), K2)

def double_round_decrypt(ciphertext, K1, K2):
    """Decrypt a block using independent round keys."""
    return decrypt_round1(decrypt_round2(ciphertext, K2), K1)

def meet_in_the_middle_attack(pairs):
    """Recover candidate (K1, K2) pairs from plaintext/ciphertext examples.

    Args:
        Plaintext/ciphertext pairs.

    Returns:
        Sorted key-pair candidates consistent with all pairs.
    """
    pairs = list(pairs)
    if not pairs:
        raise ValueError('At least one plaintext/ciphertext pair is required')

    plaintext0, ciphertext0 = pairs[0]
    forward_map = defaultdict(list)
    for K1 in range(0x10000):
        intermediate = encrypt_round1(plaintext0, K1)
        forward_map[intermediate].append(K1)

    candidates = set()
    for K2 in range(0x10000):
        intermediate = decrypt_round2(ciphertext0, K2)
        if intermediate in forward_map:
            for K1 in forward_map[intermediate]:
                candidates.add((K1, K2))

    for plaintext, ciphertext in pairs[1:]:
        candidates = {
            (K1, K2)
            for (K1, K2) in candidates
            if double_round_encrypt(plaintext, K1, K2) == ciphertext
        }
        if not candidates:
            break

    # print('Sorted candidates ', sorted(candidates))

    return sorted(candidates)


In [130]:
# =============================
# Cell 12 - Meet-in-the-middle attack
# =============================

secret_K1 = 0x1234
secret_K2 = 0xABCD
plaintext_samples = [0x0123, 0x4567, 0x89AB, 0xFEDC, 0x0F0F]
pairs_for_attack = [
    (plaintext, double_round_encrypt(plaintext, secret_K1, secret_K2))
    for plaintext in plaintext_samples
]

final_candidates = None

for count in range(1, len(pairs_for_attack) + 1):
    subset = pairs_for_attack[:count]
    candidates = meet_in_the_middle_attack(subset)
    final_candidates = candidates
    print(f'Using {count} plaintext/ciphertext pair(s) -> {len(candidates)} candidate key pair(s)')

    preview = candidates if len(candidates) <= 5 else candidates[:5]
    for K1, K2 in preview:
        print(f'  Candidate -> K1={K1:04X}, K2={K2:04X}')
    if len(candidates) > 5:
        print('  ...')

if final_candidates:
    final_count = len(final_candidates)
    if final_count == 1:
        recovered_K1, recovered_K2 = final_candidates[0]
        print('Unique key pair recovered.')
    else:
        recovered_K1, recovered_K2 = next(
            ((K1, K2) for (K1, K2) in final_candidates if (K1, K2) == (secret_K1, secret_K2)),
            final_candidates[0],
        )
        print(f'Multiple candidates remain ({final_count}). Displaying one consistent pair:')


    plaintext_to_recover = 0xFFFF

    ciphertext_example = double_round_encrypt(plaintext_to_recover, secret_K1, secret_K2)

    recovered_plaintext = double_round_decrypt(ciphertext_example, recovered_K1, recovered_K2)

    print(f'Using recovered keys K1={recovered_K1:04X}, K2={recovered_K2:04X}')
    print(f'  Decrypting ciphertext {ciphertext_example:04X} yields plaintext {recovered_plaintext:04X}')
    assert recovered_plaintext == plaintext_to_recover
else:
    print('No key pairs were recovered. Collect additional pairs and retry.')


Using 1 plaintext/ciphertext pair(s) -> 65536 candidate key pair(s)
  Candidate -> K1=0000, K2=BD0F
  Candidate -> K1=0001, K2=BD4F
  Candidate -> K1=0002, K2=BD1F
  Candidate -> K1=0003, K2=BD2F
  Candidate -> K1=0004, K2=BDCF
  ...
Using 2 plaintext/ciphertext pair(s) -> 64 candidate key pair(s)
  Candidate -> K1=1231, K2=AB4D
  Candidate -> K1=1234, K2=ABCD
  Candidate -> K1=1239, K2=ABBD
  Candidate -> K1=123C, K2=AB3D
  Candidate -> K1=12E1, K2=AB48
  ...
Using 3 plaintext/ciphertext pair(s) -> 8 candidate key pair(s)
  Candidate -> K1=1234, K2=ABCD
  Candidate -> K1=1734, K2=A3CD
  Candidate -> K1=1A34, K2=A4CD
  Candidate -> K1=1F34, K2=ACCD
  Candidate -> K1=8234, K2=3BCD
  ...
Using 4 plaintext/ciphertext pair(s) -> 4 candidate key pair(s)
  Candidate -> K1=1234, K2=ABCD
  Candidate -> K1=1734, K2=A3CD
  Candidate -> K1=1A34, K2=A4CD
  Candidate -> K1=1F34, K2=ACCD
Using 5 plaintext/ciphertext pair(s) -> 1 candidate key pair(s)
  Candidate -> K1=1234, K2=ABCD
Unique key pair r